<a href="https://colab.research.google.com/github/jaeguk-ju/ju/blob/master/C5_sunspots_A_%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9C_sol_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Sequence Modelling Question.
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv
# provided. Your neural network is expected to have an MAE
# of at least 20, with top marks going to one with an MAE
# of around 15. At the bottom is provided some testing
# code should you want to check before uploading which measures
# the MAE for you. Strongly recommend you test your model with
# this to be able to see how it performs.


import csv
import tensorflow as tf
import numpy as np
import urllib




# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

	# Your data should be loaded into 2 Python lists called time_step
	# and sunspots. They are decleared here.
    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))   # YOUR CODE HERE
        time_step.append(int(row[0]))   # YOUR CODE HERE

	# You should use numpy to create 
	# - your series from the list of sunspots
	# - your time details from the list of time steps
    series = np.array(sunspots) # YOUR CODE HERE
    time = np.array(time_step)

	# You should split the dataset into training and validation splits
	# At time 3000. So everything up to 3000 is training, and everything
	# after 3000 is validation. Write the code below to achieve that.
    split_time = 3000
    time_train = time[:split_time]# YOUR CODE HERE
    x_train = series[:split_time]# YOUR CODE HERE
    time_valid = time[split_time:]# YOUR CODE HERE
    x_valid = series[split_time:]# YOUR CODE HERE


    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000


    tf.keras.backend.clear_session()
    # You can use any random seed you want. We use 51. :)
    tf.random.set_seed(51)
    np.random.seed(51)
    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    valid_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(60, kernel_size=5, padding='causal',
                             activation='relu', input_shape=[None, 1]),
      tf.keras.layers.LSTM(60, return_sequences=True),                                  
      tf.keras.layers.LSTM(60, return_sequences=True),                                  
      tf.keras.layers.Dense(30, activation='relu'),                                        
      tf.keras.layers.Dense(10, activation='relu'),                                        
      # YOUR CODE HERE. DO NOT CHANGE THE FINAL TWO LAYERS FROM BELOW
      tf.keras.layers.Dense(1),
      # The data is not normalized, so this lambda layer helps
      # keep the MAE in line with expectations. Do not modify.
      tf.keras.layers.Lambda(lambda x: x * 400)
    ])
    opt = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=['mae'])
    checkpoint_path = 'setspot_checkpoint.ckpt'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                    save_weights_only = True, 
                                                    save_best_only=True, 
                                                    monitor='val_mae', 
                                                    verbose=1)
    model.fit(train_set, validation_data=valid_set, epochs=100,
              callbacks=[checkpoint])
    model.load_weights(checkpoint_path)



    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF5-sunspots-type-A.h5")



# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
#def model_forecast(model, series, window_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(32).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast


#window_size = # YOUR CODE HERE
#rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
#rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

#result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

## WE EXPECT AN MAE OF 15 or less for the maximum score
#score = ceil(20 - result)
#if score > 5:
#    score = 5

#print(score)


Epoch 1/100
93/93 [==============================] - 7s 44ms/step - loss: 38.0205 - mae: 38.5165 - val_loss: 20.6667 - val_mae: 21.1604

Epoch 00001: val_mae improved from inf to 21.16041, saving model to setspot_checkpoint.ckpt
Epoch 2/100
93/93 [==============================] - 3s 34ms/step - loss: 22.6314 - mae: 23.1255 - val_loss: 17.9919 - val_mae: 18.4847

Epoch 00002: val_mae improved from 21.16041 to 18.48470, saving model to setspot_checkpoint.ckpt
Epoch 3/100
93/93 [==============================] - 3s 35ms/step - loss: 19.7494 - mae: 20.2421 - val_loss: 15.5433 - val_mae: 16.0358

Epoch 00003: val_mae improved from 18.48470 to 16.03577, saving model to setspot_checkpoint.ckpt
Epoch 4/100
93/93 [==============================] - 3s 35ms/step - loss: 19.0871 - mae: 19.5792 - val_loss: 14.9894 - val_mae: 15.4807

Epoch 00004: val_mae improved from 16.03577 to 15.48068, saving model to setspot_checkpoint.ckpt
Epoch 5/100
93/93 [==============================] - 4s 38ms/step - l

In [4]:
import csv
import urllib.request

url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

# Your data should be loaded into 2 Python lists called time_step
# and sunspots. They are decleared here.
time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      print(row)
 #     sunspots.append(# YOUR CODE HERE)
#      time_step.append(# YOUR CODE HERE)


['0', '1749-01-31', '96.7']
['1', '1749-02-28', '104.3']
['2', '1749-03-31', '116.7']
['3', '1749-04-30', '92.8']
['4', '1749-05-31', '141.7']
['5', '1749-06-30', '139.2']
['6', '1749-07-31', '158.0']
['7', '1749-08-31', '110.5']
['8', '1749-09-30', '126.5']
['9', '1749-10-31', '125.8']
['10', '1749-11-30', '264.3']
['11', '1749-12-31', '142.0']
['12', '1750-01-31', '122.2']
['13', '1750-02-28', '126.5']
['14', '1750-03-31', '148.7']
['15', '1750-04-30', '147.2']
['16', '1750-05-31', '150.0']
['17', '1750-06-30', '166.7']
['18', '1750-07-31', '142.3']
['19', '1750-08-31', '171.7']
['20', '1750-09-30', '152.0']
['21', '1750-10-31', '109.5']
['22', '1750-11-30', '105.5']
['23', '1750-12-31', '125.7']
['24', '1751-01-31', '116.7']
['25', '1751-02-28', '72.5']
['26', '1751-03-31', '75.5']
['27', '1751-04-30', '94.0']
['28', '1751-05-31', '101.2']
['29', '1751-06-30', '84.5']
['30', '1751-07-31', '110.5']
['31', '1751-08-31', '99.7']
['32', '1751-09-30', '39.2']
['33', '1751-10-31', '38.7']